In [23]:
import pandas as pd
import numpy as np
import sqlite3 



conn = sqlite3.connect("acoes_data_nvidia.db")
ticker_nvidia = pd.read_sql_query("SELECT * FROM stock_data", conn)
conn.close()

display(ticker_nvidia.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [24]:
#Tratamento dos dados
ticker_data = ticker_nvidia.drop_duplicates()
ticker_data.ffill(inplace=True)  # Preenchendo valores ausentes
ticker_data.dropna()
ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])  # Convertendo para datetime

In [25]:
#Análise Descritiva
display(ticker_data.describe())

,Date,Open,High,Low,Close,Adj Close,Volume
count,1185,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1.185000e+03
mean,2022-05-09 15:47:50.886075904,32.482193,33.109483,31.804716,32.490245,32.465887,4.601158e+08
min,2020-01-02 00:00:00,5.002500,5.248500,4.517000,4.910000,4.892764,9.788400e+07
25%,2021-03-08 00:00:00,13.480000,13.673500,13.231750,13.435250,13.403188,3.255599e+08
50%,2022-05-09 00:00:00,19.962500,20.400000,19.657000,20.002501,19.964701,4.347440e+08
75%,2023-07-14 00:00:00,42.160000,42.872002,41.629002,42.238998,42.226227,5.574670e+08
max,2024-09-17 00:00:00,139.800003,140.759995,132.419998,135.580002,135.568405,1.543911e+09
std,NaN,30.528920,31.128399,29.804131,30.492050,30.497368,1.846799e+08


In [26]:
#Feature Engineering

#Porcentagem de Mudança
ticker_data['Pct Change'] = ticker_data['Close'].pct_change()

#Média Móvel
ticker_data['MA 50'] = ticker_data['Close'].rolling(window=50).mean()
ticker_data['MA 200'] = ticker_data['Close'].rolling(window=50).mean()
#Desvio Padrão
ticker_data['STD 50'] = ticker_data['Close'].rolling(window=50).std()

#Volatilidade
ticker_data['Return'] = ticker_data['Close'].pct_change()

# Calcular a volatilidade (desvio padrão dos retornos diários)
daily_volatility = ticker_data['Return'].std()

#anulizar a Volatilidade
annualized_volatility = daily_volatility * (252 ** 0.5)

display(ticker_data.describe())

,Date,Open,High,Low,Close,Adj Close,Volume,Pct Change,MA 50,MA 200,STD 50,Return
count,1185,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1.185000e+03,1184.000000,1136.000000,1136.000000,1136.000000,1184.000000
mean,2022-05-09 15:47:50.886075904,32.482193,33.109483,31.804716,32.490245,32.465887,4.601158e+08,0.003091,31.257123,31.257123,3.037603,0.003091
min,2020-01-02 00:00:00,5.002500,5.248500,4.517000,4.910000,4.892764,9.788400e+07,-0.184521,6.337560,6.337560,0.249896,-0.184521
25%,2021-03-08 00:00:00,13.480000,13.673500,13.231750,13.435250,13.403188,3.255599e+08,-0.016053,13.593592,13.593592,1.036165,-0.016053
50%,2022-05-09 00:00:00,19.962500,20.400000,19.657000,20.002501,19.964701,4.347440e+08,0.003250,20.399625,20.399625,1.930992,0.003250
75%,2023-07-14 00:00:00,42.160000,42.872002,41.629002,42.238998,42.226227,5.574670e+08,0.022723,41.658125,41.658125,3.058084,0.022723
max,2024-09-17 00:00:00,139.800003,140.759995,132.419998,135.580002,135.568405,1.543911e+09,0.243696,120.899200,120.899200,17.981939,0.243696
std,NaN,30.528920,31.128399,29.804131,30.492050,30.497368,1.846799e+08,0.034496,27.958270,27.958270,3.390624,0.034496


In [27]:
import plotly.graph_objs as go


fig_ = go.Figure()

# Adicionando os dados ao gráfico
fig_.add_trace(go.Scatter(
    x=ticker_nvidia['Date'],  # Coluna 'Date' no eixo x
    y=ticker_nvidia['Open'],  # Coluna 'Close' no eixo y
    mode='lines',  # Modo de exibição como linhas
    name='Abertura',  # Nome da linha no gráfico
    line=dict(color='blue')
))

fig_.add_trace(go.Scatter(
    x=ticker_nvidia['Date'],  # Coluna 'Date' no eixo x
    y=ticker_nvidia['Close'],  # Coluna 'Close' no eixo y
    mode='lines',  # Modo de exibição como linhas
    name='Fechamento',  # Nome da linha no gráfico
    line=dict(color='yellow')
))


fig_.add_trace(go.Scatter(
    x=ticker_nvidia['Date'],  # Coluna 'Date' no eixo x
    y=ticker_nvidia['High'],  # Coluna 'Close' no eixo y
    mode='lines',  # Modo de exibição como linhas
    name='Preço mais alto',  # Nome da linha no gráfico
    line=dict(color='red')
))

fig_.add_trace(go.Scatter(
    x=ticker_nvidia['Date'],  # Coluna 'Date' no eixo x
    y=ticker_nvidia['Low'],  # Coluna 'Close' no eixo y
    mode='lines',  # Modo de exibição como linhas
    name='Preço mais Baixo',  # Nome da linha no gráfico
    line=dict(color='green')
))

# Atualizar o layout do gráfico
fig_.update_layout(
    title='Preço de Fechamento da Ação NVIDIA',
    xaxis_title='Data',
    yaxis_title='Preço de Fechamento',
    legend_title='Ação'
)

# Mostrar o gráfico
fig_.show()

In [29]:
# # Adicionar linha de média móvel
# fig_Ma = go.Figure()
# fig_Ma.add_trace(go.Scatter(
#     x=ticker_data["Close"],
#     y=ticker_data['MA 50'],
#     mode='lines',
#     name='Média Móvel (50 dias)',
#     line=dict(color='orange')
# ))


# Criar gráfico
fig = go.Figure()

# Adicionar linha de preços de fechamento
fig.add_trace(go.Scatter(
    x=ticker_data["Date"],
    y=ticker_data['Close'],
    mode='lines',
    name='Preço de Fechamento',
    line=dict(color='blue')
))

# Adicionar linha de média móvel
fig.add_trace(go.Scatter(
    x=ticker_data["Date"],
    y=ticker_data['MA 50'],
    mode='lines',
    name='Média Móvel (50 dias)',
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=ticker_data["Date"],
    y=ticker_data['MA 200'],
    mode='lines',
    name='Média Móvel (200 dias)',
    line=dict(color='green')
))

# Adicionar linhas para o desvio padrão
fig.add_trace(go.Scatter(
    x=ticker_data["Date"],
    y=ticker_data['MA 50'] + ticker_data['STD 50'],
    mode='lines',
    name='Média Móvel + Desvio Padrão',
    line=dict(color='red', dash='dash')
))


fig.add_trace(go.Scatter(
    x=ticker_data["Date"],
    y=ticker_data['MA 50'] - ticker_data['STD 50'],
    mode='lines',
    name='Média Móvel - Desvio Padrão',
    line=dict(color='red', dash='dash')
))

# Configurar layout
fig.update_layout(
    title='Preço de Fechamento, Média Móvel e Desvio Padrão',
    xaxis_title='Data',
    yaxis_title='Preço',
    legend_title='Séries'
)

# Mostrar gráfico
fig.show()